In [35]:
# objętość prostaty i jej stref
# objętość zmian na sekwencjach, gdzie zostały obrysowane (niezależnie T2W, ADC, DWI, DCE)
# średnia wartość ADC dla zmian
# mapowanie obrysów prostaty (strefa PZ, TZ) na ADC i obliczenie średniego ADC w każdej z nich - do obliczenia średniego ADC należy pominąć obszary zmian
# obwód prostaty mierzony na warstwie, gdzie maska prostaty ma największą powierzchnię

#good example to test registration
# https://github.com/InsightSoftwareConsortium/ITKElastix/blob/main/examples/ITK_Example08_SimpleTransformix.ipynb

In [36]:
import numpy as np
import SimpleITK as sitk
import mdai
import pandas as pd
import numpy as np
import cv2
import pydicom
import os
import multiprocessing as mp
import functools
from functools import partial
import mdai
import math
import time
import mainFuncs
import itertools
import torch
from os import path as pathOs
import more_itertools
from os.path import basename, dirname, exists, isdir, join, split
import shutil
from mainFuncs import getLabelsAbbrev
import os
from radiomics import firstorder, getTestCase, glcm, glrlm, glszm, imageoperations, shape,shape2D
import SimpleITK as sitk
import mdai
import pandas as pd
import numpy as np
import cv2
import pydicom
import os
import multiprocessing as mp
import functools
from functools import partial
import mdai
import math
import time
import itertools
from pydicom.fileset import FileSet
from os import path as pathOs
from pathlib import Path
import toolz
from toolz.curried import pipe, map, filter, get
from toolz import curry
from os.path import basename, dirname, exists, isdir, join, split
import nnunetv2

import nnunet.elastixRegister as elastixRegister
from nnunet.elastixRegister import reg_a_to_b,reg_a_to_b_itk
import tempfile
import shutil
import re
from toolz.itertoolz import groupby
from toolz import curry
# import multiprocess

In [37]:
import pandas as pd
ihor_csv_dir= '/home/sliceruser/workspaces/konwersjaJsonData/metaDataProject/SR_per_radiologist_full.csv'
prost_volumes_calculated= '/workspaces/konwersjaJsonData/explore/prost_volumes.csv'
ihor_csv_df=pd.read_csv(ihor_csv_dir)
prost_volumes_calculated_df= pd.read_csv(prost_volumes_calculated)
prost_volumes_calculated_df['patient_id']=prost_volumes_calculated_df['number']
prost_volumes_calculated_df['patient_id'] = prost_volumes_calculated_df['patient_id'].astype(float)
prost_volumes_calculated_df


,Unnamed: 0,study_id,number,volume,name,patient_id
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,36.124365,pg,234.0
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,0.762803,afs,234.0
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,2.305811,cz,234.0
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,8.611260,pz,234.0
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,12.364365,sv,234.0
...,...,...,...,...,...,...
2463,2463,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.440049,pz,1188.0
2464,2464,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.838760,sv,1188.0
2465,2465,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.711142,sv,1188.0
2466,2466,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.585068,tz,1188.0


In [38]:
prost_volumes_calculated_df_calc=prost_volumes_calculated_df[['patient_id', 'volume', 'name']]
prost_volumes_calculated_df_calc=prost_volumes_calculated_df_calc.loc[prost_volumes_calculated_df_calc['name'] == 'pg'][['patient_id', 'volume']]
prost_volumes_calculated_df_calc=prost_volumes_calculated_df_calc.sort_values(by=['patient_id'])
ihor_csv_df=ihor_csv_df.sort_values(by=['patient_id'])
ihor_csv_df


,_id,adequate_fov_dce,adequate_fov_dwi,adequate_fov_t2w,adequate_resolution_dce,adequate_resolution_dwi,adequate_resolution_t2w,adequate_slice_thickness_dce,adequate_slice_thickness_dwi,adequate_slice_thickness_t2w,...,total_time_dce,uniformity_pz,visual_assessment_dce,visual_assessment_dwi,visual_assessment_t2w,visual_params_dce,visual_params_dwi,visual_params_t2w,xnat_project_id,zaxis_t2w
312,62b412649953680001e68e71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
492,62d2cd8099536800013e1a4d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
1244,63452e6899536800013e8ae0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
0,628a95d79953680001d319a6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
20,6295cc2299536800011be602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,635eb1d099536800013ea89c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
1315,635046d399536800013e96af,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
1471,637228d399536800013ebc16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN
1316,63504aeb99536800013e96bd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,HETEROGENEOUS,NaN,NaN,NaN,NaN,NaN,NaN,AI4AR_PCa,NaN


In [39]:
a=ihor_csv_df[['patient_id','prostate_volume_calculated']].drop_duplicates(subset=['patient_id']).sort_values(by=['patient_id'])

prost_volumes_calculated_df

,Unnamed: 0,study_id,number,volume,name,patient_id
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,36.124365,pg,234.0
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,0.762803,afs,234.0
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,2.305811,cz,234.0
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,8.611260,pz,234.0
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,12.364365,sv,234.0
...,...,...,...,...,...,...
2463,2463,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.440049,pz,1188.0
2464,2464,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.838760,sv,1188.0
2465,2465,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.711142,sv,1188.0
2466,2466,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.585068,tz,1188.0


In [40]:
b=prost_volumes_calculated_df_calc
merged=pd.merge(a, b, on=["patient_id"])
diffs=merged['prostate_volume_calculated'].to_numpy()-merged['volume'].to_numpy()
diffs=np.abs(diffs)
mean= np.round(np.mean(diffs),2)
var = np.round(np.std(diffs),2)
print(f"mean {mean} var {var} mean value {np.mean(merged['volume'])} ")

mean 8.77 var 8.27 mean value 55.51570739868619 


In [41]:
np.unique(prost_volumes_calculated_df['name'])

array(['afs', 'cz', 'pg', 'pz', 'sv', 'tz', 'ur'], dtype=object)

## check if parts of the prostate is always smaller then full prostate

In [42]:
# objętość prostaty i jej stref
# check if parts of the prostate is always smaller then full prostate
def groupByMaster(rowws):
    grouped_by_master= groupby(lambda row : row[1]['patient_id'],rowws)
    return dict(grouped_by_master).items()

def get_vol(row,colName):
    if(  row[1]['name'] ==colName ):
        return row[1]['volume']
    return 0.0

def get_vols(colName,listRows):
    """
    extracting all paths of intrest from row
    """
    res=  list(map( partial(get_vol,colName=colName),listRows ))
    return np.max(res)


def iterGroups(groupTuple,anatomy_cols, prostate_col_name):
    masterOlds, listRows= groupTuple
    #first find all of the lesion paths and anatomy paths
    anatomy_vols=toolz.pipe(anatomy_cols
                ,map(partial(get_vols,listRows=listRows))
                ,list) 
    prost_vol= get_vols(prostate_col_name,listRows)
    if(prost_vol==0):
        return (masterOlds,-1.0)
    res= list(map( lambda part_vol : part_vol>prost_vol ,anatomy_vols ))
    return (masterOlds,np.sum(res) )


def are_parts_smaller(sourceFrame,anatomy_cols,prost_name):
    vals= toolz.pipe(sourceFrame.iterrows()
                            ,groupByMaster
                            ,list
                            ,map(partial(iterGroups,anatomy_cols=anatomy_cols,prostate_col_name=prost_name ))
                            ,list                                                        )



    return vals
 
names= ['afs', 'cz', 'pz', 'sv', 'tz', 'ur']
prost_name= 'pg'
bi=are_parts_smaller(prost_volumes_calculated_df,names,prost_name )
values=list(map(lambda el: el[1] ,bi))
np.sum(values)#if sum is 0 we have no cases of part bigger then full prostate 

0

## check is lesion dims and lesion volumes are proportional

In [43]:
# objętość zmian na sekwencjach, gdzie zostały obrysowane (niezależnie T2W, ADC, DWI, DCE)
#lesion_dim_short	lesion_dim_vertical
ihor_for_lesion_check_dir = '/home/sliceruser/workspaces/konwersjaJsonData/metaDataProject/AI4AR_PCa_radiological_full.csv'
ihor_for_lesion_check = pd.read_csv(ihor_for_lesion_check_dir)
ihor_for_lesion_check=ihor_for_lesion_check[['patient_id', 'lesion_id', 'rad_hash', 'lesion_dim_long','lesion_dim_max_calculated','lesion_dim_short','lesion_dim_vertical']]
ihor_for_lesion_check



# merged=pd.merge(a, b, on=["patient_id"])

,patient_id,lesion_id,rad_hash,lesion_dim_long,lesion_dim_max_calculated,lesion_dim_short,lesion_dim_vertical
0,1,1,U_OjmNZ8,0.0,16.0,16,0.0
1,1,1,U_8ZmM76,0.0,29.0,29,0.0
2,1,1,U_OrjnrO,31.0,31.0,11,20.0
3,3,1,U_8ekGLQ,0.0,9.0,9,0.0
4,3,1,U_OjmNZ8,0.0,7.0,7,0.0
...,...,...,...,...,...,...,...
1063,3359,1,U_8ekGLQ,0.0,20.0,20,0.0
1064,3359,1,U_8ZmM76,0.0,17.0,17,0.0
1065,3360,1,U_8ekGLQ,0.0,20.0,20,0.0
1066,3360,1,U_OjmNZ8,0.0,17.0,17,0.0


In [44]:
lesion_volumes_dir='/workspaces/konwersjaJsonData/explore/lesion_volumes.csv'
lesion_volumes = pd.read_csv(lesion_volumes_dir)
lesion_volumes['patient_id']=lesion_volumes['number']
lesion_volumes['rad_hash']=lesion_volumes['annotator_id']
lesion_volumes['lesion_id']=list(map(lambda el: int(el.replace('lesion', '')) ,lesion_volumes['lesion_name'].to_numpy()))
lesion_volumes


,Unnamed: 0,study_id,number,modality,volume,lesion_name,annotator_id,patient_id,rad_hash,lesion_id
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,2.378320,lesion1,U_8ZmM76,234,U_8ZmM76,1
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,2.775674,lesion1,U_8ekGLQ,234,U_8ekGLQ,1
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,2.143389,lesion1,U_OjmNZ8,234,U_OjmNZ8,1
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,0.835312,lesion2,U_8ZmM76,234,U_8ZmM76,2
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,t2w,0.522070,lesion2,U_8ekGLQ,234,U_8ekGLQ,2
...,...,...,...,...,...,...,...,...,...,...
3981,3981,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,adc,0.760860,lesion1,U_8ZmM76,1188,U_8ZmM76,1
3982,3982,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,adc,1.299802,lesion1,U_8ekGLQ,1188,U_8ekGLQ,1
3983,3983,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,adc,0.412132,lesion1,U_OjmNZ8,1188,U_OjmNZ8,1
3984,3984,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,adc,1.363207,lesion2,U_8ekGLQ,1188,U_8ekGLQ,2


In [45]:
merged=pd.merge(lesion_volumes, ihor_for_lesion_check, on=["patient_id", "rad_hash","lesion_id"])
mergedd=merged[['lesion_dim_long','lesion_dim_max_calculated','lesion_dim_short','lesion_dim_vertical',"volume"]]
mergedd.corr(method ='pearson')

,lesion_dim_long,lesion_dim_max_calculated,lesion_dim_short,lesion_dim_vertical,volume
lesion_dim_long,1.000000,0.206714,0.125640,0.893632,0.285599
lesion_dim_max_calculated,0.206714,1.000000,0.990331,0.230209,0.593045
lesion_dim_short,0.125640,0.990331,1.000000,0.158530,0.579112
lesion_dim_vertical,0.893632,0.230209,0.158530,1.000000,0.316330
volume,0.285599,0.593045,0.579112,0.316330,1.000000


In [46]:
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            tmp = df[df[r].notnull() & df[c].notnull()]
            pvalues[r][c] = round(pearsonr(tmp[r], tmp[c])[1], 4)
    return pvalues

calculate_pvalues(mergedd)

,lesion_dim_long,lesion_dim_max_calculated,lesion_dim_short,lesion_dim_vertical,volume
lesion_dim_long,0.0,0.0,0.0,0.0,0.0
lesion_dim_max_calculated,0.0,0.0,0.0,0.0,0.0
lesion_dim_short,0.0,0.0,0.0,0.0,0.0
lesion_dim_vertical,0.0,0.0,0.0,0.0,0.0
volume,0.0,0.0,0.0,0.0,0.0


## check is adc  related to pirads

In [47]:
def string_pirads_to_int(el):
    try:
        return int(el)
    except:
        return -1    

In [48]:
mean_adcs_dir='/workspaces/konwersjaJsonData/explore/mean_adcs.csv'
mean_adcs_lesions_df = pd.read_csv(mean_adcs_dir)
mean_adcs_lesions_df['patient_id']=mean_adcs_lesions_df['number']
mean_adcs_lesions_df['rad_hash']=mean_adcs_lesions_df['annotator_id']
mean_adcs_lesions_df['lesion_id']=list(map(lambda el: int(el.replace('lesion', '')) ,mean_adcs_lesions_df['lesion_name'].to_numpy()))


ihor_for_lesion_check = pd.read_csv(ihor_for_lesion_check_dir)
ihor_for_lesion_check['lesion_pirads']= list(map(string_pirads_to_int,ihor_for_lesion_check['lesion_pirads'].to_numpy() ))
merged=pd.merge(mean_adcs_lesions_df, ihor_for_lesion_check, on=["patient_id", "rad_hash","lesion_id"])
mergedd=merged[['lesion_pirads',"mean_adc"]]
calculate_pvalues(mergedd)
# mergedd

,lesion_pirads,mean_adc
lesion_pirads,0.0,0.0
mean_adc,0.0,0.0


In [49]:
# średnia wartość ADC dla zmian

## check is anatomy mean adc is consistent among anatomic areas and is in adc calculated volume volume of anatomic region is smaller if some lesion is present there

In [50]:
mean_adc_anatomy_dir= '/workspaces/konwersjaJsonData/explore/mean_adcs_per_anatomy.csv'
mean_anatomy_df = pd.read_csv(mean_adc_anatomy_dir)
mean_anatomy_df

,Unnamed: 0,id,pg_noSeg,pg_noSeg_volume,afs_noSeg,afs_noSeg_volume,cz_noSeg,cz_noSeg_volume,pz_noSeg,pz_noSeg_volume,sv_l_noSeg,sv_l_noSeg_volume,sv_r_noSeg,sv_r_noSeg_volume,tz_noSeg,tz_noSeg_volume,ur_noSeg,ur_noSeg_volume
0,0,234,203.936813,30.466087,328.752808,0.697455,206.035980,1.553422,179.470779,5.547935,485.342651,12.110349,374.55466,12.23715865761135,205.94064,22.667275751793046,129.93501,0.9193720234992983
1,1,666,243.773041,33.141972,209.268829,1.845679,239.085495,5.657407,261.860443,8.626542,348.227203,6.219135,334.7391,4.373456315114931,239.9036,17.01234383127276,,
2,2,3,226.986877,61.056926,258.996307,2.374754,182.704575,11.072648,245.130585,35.363805,210.837067,7.953995,206.34465,4.492004391559978,205.40895,10.90097881009141,212.97241,1.7453010693322206
3,3,17,1218.825000,57.222988,1030.335766,3.919775,1274.984615,1.859747,1182.323296,18.053853,1445.034404,12.474611,1438.1340206185566,13.876574570324529,1262.8775510204082,32.245153692279885,1083.676923076923,1.8597471073630811
4,4,26,1048.197674,57.254684,1249.956522,0.729157,792.740741,0.855967,738.772131,19.338514,846.012474,15.248894,1005.7799145299145,14.83676185888458,1214.869982547993,36.331045064704554,902.8333333333334,0.1902148956267254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,286,1184,1321.100089,128.637275,1167.253333,2.145862,1303.935185,3.090041,1307.520000,9.298735,1273.066964,6.408975,1280.6008403361345,6.80953546373751,1325.669095477387,113.87374430956005,1188.857142857143,0.4005609096316182
287,287,1221,1015.998694,58.984697,998.804878,1.052840,968.617021,4.827655,951.985337,8.756544,575.075188,3.415309,825.5345911949686,4.082963344465608,1034.1501746216532,44.11654733202462,1038.6296296296296,0.6933333981168014
288,288,1218,1092.137380,80.598576,1242.784615,1.859747,786.276596,1.344740,898.487026,14.334358,239.872727,3.147264,490.58870967741933,3.547825124544186,1137.157433056325,61.97249370776377,1131.8070175438597,1.6308550975727307
289,289,1096,953.712723,78.481325,1056.984375,3.662271,1012.006329,4.520616,674.146283,11.930992,1021.767647,9.727908,970.5873605947955,7.696491550650489,1002.4205986808727,56.393252216848005,841.8024691358024,2.3175309130211508


In [53]:
print(mean_anatomy_df.mean())
print(mean_anatomy_df.std())


Unnamed: 0            145.000000
id                    890.900344
pg_noSeg             1060.191128
pg_noSeg_volume        53.543674
afs_noSeg            1090.098540
afs_noSeg_volume        2.226380
cz_noSeg             1015.743403
cz_noSeg_volume         3.840271
pz_noSeg              942.278791
pz_noSeg_volume        12.484405
sv_l_noSeg            892.624073
sv_l_noSeg_volume       6.965044
dtype: float64
Unnamed: 0            84.148678
id                   924.381844
pg_noSeg             216.542521
pg_noSeg_volume       32.361143
afs_noSeg            242.470324
afs_noSeg_volume       1.849236
cz_noSeg             289.296257
cz_noSeg_volume        3.103157
pz_noSeg             286.096852
pz_noSeg_volume        5.403694
sv_l_noSeg           409.375959
sv_l_noSeg_volume      3.858473
dtype: float64


The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


In [54]:
prost_volumes_calculated_df

,Unnamed: 0,study_id,number,volume,name,patient_id
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,36.124365,pg,234.0
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,0.762803,afs,234.0
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,2.305811,cz,234.0
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,8.611260,pz,234.0
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,12.364365,sv,234.0
...,...,...,...,...,...,...
2463,2463,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.440049,pz,1188.0
2464,2464,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.838760,sv,1188.0
2465,2465,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.711142,sv,1188.0
2466,2466,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.585068,tz,1188.0


In [55]:
prost_volumes_calculated_df['patient_id']=prost_volumes_calculated_df['number'] 
mean_anatomy_df['patient_id']=mean_anatomy_df['id'] 

merged=pd.merge(prost_volumes_calculated_df, mean_anatomy_df, on=["patient_id"])
merged

,Unnamed: 0_x,study_id,number,volume,name,patient_id,Unnamed: 0_y,id,pg_noSeg,pg_noSeg_volume,...,pz_noSeg,pz_noSeg_volume,sv_l_noSeg,sv_l_noSeg_volume,sv_r_noSeg,sv_r_noSeg_volume,tz_noSeg,tz_noSeg_volume,ur_noSeg,ur_noSeg_volume
0,0,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,36.124365,pg,234,0,234,203.936813,30.466087,...,179.470779,5.547935,485.342651,12.110349,374.55466,12.23715865761135,205.94064,22.667275751793046,129.93501,0.9193720234992983
1,1,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,0.762803,afs,234,0,234,203.936813,30.466087,...,179.470779,5.547935,485.342651,12.110349,374.55466,12.23715865761135,205.94064,22.667275751793046,129.93501,0.9193720234992983
2,2,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,2.305811,cz,234,0,234,203.936813,30.466087,...,179.470779,5.547935,485.342651,12.110349,374.55466,12.23715865761135,205.94064,22.667275751793046,129.93501,0.9193720234992983
3,3,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,8.611260,pz,234,0,234,203.936813,30.466087,...,179.470779,5.547935,485.342651,12.110349,374.55466,12.23715865761135,205.94064,22.667275751793046,129.93501,0.9193720234992983
4,4,1.3.12.2.1107.5.2.41.69644.3000002001140527227...,234,12.364365,sv,234,0,234,203.936813,30.466087,...,179.470779,5.547935,485.342651,12.110349,374.55466,12.23715865761135,205.94064,22.667275751793046,129.93501,0.9193720234992983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,2463,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.440049,pz,1188,290,1188,971.801573,36.267640,...,866.625668,17.785093,480.386293,10.176497,983.1915584415584,9.764364720850152,1084.7838427947597,14.519737149835615,1166.6296296296296,0.8559670372173834
2231,2464,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.838760,sv,1188,290,1188,971.801573,36.267640,...,866.625668,17.785093,480.386293,10.176497,983.1915584415584,9.764364720850152,1084.7838427947597,14.519737149835615,1166.6296296296296,0.8559670372173834
2232,2465,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,10.711142,sv,1188,290,1188,971.801573,36.267640,...,866.625668,17.785093,480.386293,10.176497,983.1915584415584,9.764364720850152,1084.7838427947597,14.519737149835615,1166.6296296296296,0.8559670372173834
2233,2466,1.3.6.1.4.1.45002.1.3.9999.9889643,1188,17.585068,tz,1188,290,1188,971.801573,36.267640,...,866.625668,17.785093,480.386293,10.176497,983.1915584415584,9.764364720850152,1084.7838427947597,14.519737149835615,1166.6296296296296,0.8559670372173834


In [62]:
def groupByMaster(rowws):
    grouped_by_master= groupby(lambda row : row[1]['patient_id'],rowws)
    return dict(grouped_by_master).items()

def my_to_float(el):
    try: 
        return float(el)
    except:
        return 0.0

def get_vol(row,colName):
    if(  row[1]['name'] ==colName ):
        return row[1]['volume']-my_to_float(row[1][f"{colName}_noSeg_volume"])
    return 0.0

def get_vols(colName,listRows):
    """
    extracting all paths of intrest from row
    """
    res=  list(map( partial(get_vol,colName=colName),listRows ))
    return np.max(res)


def iterGroups(groupTuple,anatomy_cols, prostate_col_name):
    masterOlds, listRows= groupTuple
    #first find all of the lesion paths and anatomy paths
    anatomy_vols=toolz.pipe(anatomy_cols
                ,map(partial(get_vols,listRows=listRows))
                ,list) 
    return (masterOlds,np.mean(anatomy_vols) )


def are_parts_smaller(sourceFrame,anatomy_cols,prost_name):
    vals= toolz.pipe(sourceFrame.iterrows()
                            ,groupByMaster
                            ,list
                            ,map(partial(iterGroups,anatomy_cols=anatomy_cols,prostate_col_name=prost_name ))
                            ,list                                                        )



    return vals
 
names= ['afs', 'cz', 'pz', 'sv_l','sv_r', 'tz', 'ur']
prost_name= 'pg'
bi=are_parts_smaller(merged,names,prost_name )
values=list(map(lambda el: el[1] ,bi))
np.mean(values)#if sum is 0 we have no cases of part bigger then full prostate 

0.9263404444084311

In [51]:
# mapowanie obrysów prostaty (strefa PZ, TZ) na ADC i obliczenie średniego ADC w każdej z nich - do obliczenia średniego ADC należy pominąć obszary zmian


## find is there correletion between volumeand dime from ihor and max perimeters 


In [52]:
perimeters_dir='/workspaces/konwersjaJsonData/explore/max_circs.csv'
perimeters_df = pd.read_csv(perimeters_dir)
perimeters_df['patient_id']=perimeters_df['id']

merged=pd.merge(perimeters_df, ihor_csv_df, on=["patient_id"])

mergedddd=merged[['pg_noSeg','prostate_dim_h','prostate_dim_l',	'prostate_dim_w',	'prostate_volume_approx_ratio',	'prostate_volume_calculated']]
calculate_pvalues(mergedddd)

An input array is constant; the correlation coefficient is not defined.


,pg_noSeg,prostate_dim_h,prostate_dim_l,prostate_dim_w,prostate_volume_approx_ratio,prostate_volume_calculated
pg_noSeg,0.0,0.0,0.0,0.0,NaN,0.0
prostate_dim_h,0.0,0.0,0.0,0.0,NaN,0.0
prostate_dim_l,0.0,0.0,0.0,0.0,NaN,0.0
prostate_dim_w,0.0,0.0,0.0,0.0,NaN,0.0
prostate_volume_approx_ratio,NaN,NaN,NaN,NaN,NaN,NaN
prostate_volume_calculated,0.0,0.0,0.0,0.0,NaN,0.0
